In [2]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate
import jax.numpy as jnp

import ctrl_sandbox.mujoco_funcs as mj_funcs
import ctrl_sandbox.mjcf_models as mj_models
import ctrl_sandbox.visualize_mj_funcs as mj_vis

import ctrl_sandbox.dyn_functions as dyn
import ctrl_sandbox.visualize_dyn_funcs as vis_dyn
import ctrl_sandbox.gen_ctrl_funcs as gen_ctrl
import ctrl_sandbox.analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=8, suppress=True, linewidth=100)


In [64]:
dt = 0.01
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
d_bar = h_bar/2
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)

In [65]:
mjcf_model  = mj_models.create_MJCF_double_pend_fa_dev()
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)
nu = mj_model.nu
nx = mj_model.nv    
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)
print(mj_data.ctrl)

[0. 0.]
[0. 0.]
[0. 0.]


In [66]:
dpend_sys = dyn.double_pend_rel_dyn(g=9.81, m1=m_bar, moi1=moi, d1=d_bar, l1=h_bar, 
                                            m2=m_bar, moi2=moi, d2=d_bar, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=True)
disc_dyn_func = lambda x, u: integrate.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [67]:
x_init = np.array([0.0,0.0,0.0,0.0])
u_vec = np.array([0.0, 0.0]) 
xu_lin_k = jnp.array(np.concatenate((x_init, u_vec)))

mj_data.ctrl = u_vec
mujoco.mj_forward(mj_model, mj_data)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps=1e-6, flg_centered=False)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, xu_lin_k, len(x_init))

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.99937059  0.00062675  0.00999809 -0.00000001]
 [ 0.00083734  0.99770092  0.00000221  0.00999512]
 [-0.12585185  0.12528918  0.99938927  0.00041736]
 [ 0.16738634 -0.45962222  0.00077924  0.99797894]]
A dyn: 
 [[ 0.9993705   0.00062697  0.0099979   0.00000209]
 [ 0.00083753  0.99770015  0.00000279  0.00999233]
 [-0.12586522  0.12533367  0.9993705   0.00062697]
 [ 0.16742484 -0.459775    0.00083753  0.99770015]]
B mujoco: 
 [[ 0.00008537 -0.00021321]
 [-0.00021314  0.00068205]
 [ 0.01706701 -0.0426164 ]
 [-0.04261016  0.13633368]]
B dyn: 
 [[ 0.00008539 -0.00021321]
 [-0.00021321  0.00068209]
 [ 0.01707094 -0.04262316]
 [-0.04262316  0.13635914]]


In [70]:
x_vec = np.array([3.1, 0.1, 0.0, 0.0])
u_vec = np.array([10.0,10.0])
xu_k = jnp.concatenate((x_vec, u_vec))
mj_funcs.set_mj_ctrl_vec(mj_data, u_vec)
mj_funcs.set_mj_state_vec(mj_data, x_vec)
mujoco.mj_forward(mj_model, mj_data)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps=1e-6, flg_centered=True)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, xu_k, len(x_vec))

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.99957455  0.0006732   0.01006634  0.00008042]
 [ 0.001081    0.99822208 -0.00020739  0.00985319]
 [-0.08454954  0.13394367  1.01714567  0.01664101]
 [ 0.21577407 -0.3529352  -0.05109887  0.96937231]]
A dyn: 
 [[ 0.99957454  0.00067336  0.01010536  0.00008076]
 [ 0.00108115  0.9982216  -0.00031057  0.00985184]
 [-0.08455643  0.13397464  1.0210048   0.01667596]
 [ 0.2158029  -0.3530341  -0.06138198  0.9692345 ]]
B mujoco: 
 [[ 0.00002544 -0.00007974]
 [-0.00012933  0.00029366]
 [ 0.00691697 -0.01578229]
 [-0.02084213  0.05849685]]
B dyn: 
 [[ 0.0000446  -0.00007966]
 [-0.00008082  0.00029406]
 [ 0.00889476 -0.01577491]
 [-0.0161259   0.05853998]]
